# Detailed usage

In [1]:
from cf_xarray.units import units  # isort:skip
import pint_xarray  # isort:skip

pint_xarray.unit_registry = units  # isort:skip
import ocean_data_gateway as odg
import pandas as pd
import xarray as xr
import numpy as np
pd.set_option('display.max_rows', 5)

## General Options

In [2]:
kw = {
    "min_lon": -124.0,
    "max_lon": -123.0,
    "min_lat": 39.0,
    "max_lat": 40.0,
    "min_time": '2021-4-1',
    "max_time": '2021-4-2',
}

### Parallel

You can control readers individually as needed. For example, you could input the keyword `parallel`, which every reader accepts, per individual reader (in case you want different values for different readers), or you can input it for all readers by including it in `kwargs` generally. It runs in parallel using the `joblib` `Parallel` and `delayed` modules with `multiprocesses` — running loops on different cores.

In [3]:
kwargs = {
          'kw': kw, 
          'approach': 'region',
          'parallel': True,    
          'erddap': {
                           'known_server': 'ioos',
#                            'parallel': False,
                           'variables': 'salinity',
          },
          'axds': {'catalog_name': None,
#                          'parallel': True,
                         'axds_type': 'platform2',
                         'variables': 'Salinity'},
          }
data = odg.Gateway(**kwargs)

### Reader Choice

Your reader choices can be selected as follows, where `odg.erddap` connects to ERDDAP servers, the `odg.axds` connects to Axiom databases, and the `odg.local` enables easy local file read-in. If you don't input any reader, it will use all of them. Alternatively you can input some subset.

In [4]:
readers = [odg.erddap,
           odg.axds,
           odg.local]

Use only ERDDAP reader and Axiom reader:

In [5]:
data = odg.Gateway(kw=kw, approach='region', 
                   readers=[odg.erddap,
                            odg.axds])

## Region

Search by time/space region.

### All variables

Don't input anything with the `variables` keyword, or use `'variables': None`:

In [6]:
kwargs = {
          'kw': kw, 
          'approach': 'region',
          'readers': [odg.erddap,
                      odg.axds],
          'variables': None
}
data = odg.Gateway(**kwargs)

### By variables(s)

If no `variables` are specified for a given reader (for a `region` search), datasets with any variables will be returned from a search.

However, if you want to specify a variable or variables, keep in mind that different readers have different names for variables, which is why you can't just input a variable name for all the readers. 

This is only relevant for the ERDDAP and Axiom readers currently (it will retain all variables in local files). The Axiom reader of type `platform2` will search by variable where the available variable names are specified, and of type `layer_group`, the `query` method will be used for variable searching.

Let's say you want to search for salinity. You can input the base of the word as `variables` ("sal" or "salinity" but not "salt" since the checker searches for matches with the whole input variable name and "salt" isn't used for any variable names) and the code will make sure it exactly matches a known variable name. If it cannot match, it will throw an error with suggestions. This is not done automatically since for example "soil_salinity" matches for "salinity". You need to do this for each `known_server` for the `erddap` reader separately, and specific variables will only be used to filter for the `axds` reader for `axds_type='platform2'`. Any variable names can be input for the `axds` reader for `axds_type='layer_group'`.

You can first check each reader/server individually to determine the variable names you want to use:

In [14]:
# IOOS ERDDAP
server = 'http://erddap.sensors.ioos.us/erddap'
odg.search_variables(server, 'sal')

,count
variable,
salinity,954
salinity_qc,954
...,...
sea_water_practical_salinity_4161sc_a_qc_agg,1
sea_water_practical_salinity_10091sc_a,1


In [12]:
# Coastwatch ERDDAP
server = 'http://coastwatch.pfeg.noaa.gov/erddap'
odg.search_variables(server, 'sal')

,count
variable,
salinity,73
salt,4
...,...
stop_salinity,1
ts_salinity,1


In [13]:
# axds 'platform2'
odg.search_variables('axds', 'sal')

,count
variable,
Salinity,3204
Soil Salinity,622


Once you have selected variables that match, the code won't complain anymore.

In [15]:
kwargs = {
          'kw': kw, 
          'approach': 'region',
          'readers': [odg.erddap,
                      odg.axds],

          'erddap': {
                          'known_server': ['coastwatch','ioos'],
                           'variables': [['salinity', 'sea_water_salinity'],
                                         ['salinity', 'sea_water_practical_salinity']]
          },
          'axds': {
                          'axds_type': ['platform2','layer_group'],
                         'variables': ['Salinity',
                                       'salinity'  # this one can be called anything that might make a match
                                      ]},
}

data = odg.Gateway(**kwargs)

In [34]:
data.dataset_ids

['d359748a-fe78-11e7-8128-0023aeec7b98',
 '5104d464-8a30-4720-aeb7-57e801844e6e',
 '3261285c-e3c9-45fd-b777-e6d681a3eaad']

### Other actions with variables

Demonstrated here are the available functions to work with variables. Currently the ways to call the individiual libraries aren't pretty but they'll work. Note that the number of times a variable is used in the system is also included under "count" to see what the popular names are (many are not widely used). 


#### All available variables

Return all variables for the two ERDDAP `known_server`s, then for the Axiom reader `axds_type='platform2'`.

In [20]:
# IOOS ERDDAP
server = 'http://erddap.sensors.ioos.us/erddap'
odg.all_variables(server)

,count
variable,
air_pressure,4028
air_pressure_10011met_a,2
...,...
wmo_id,954
z,37377


In [21]:
# Coastwatch ERDDAP
server = 'http://coastwatch.pfeg.noaa.gov/erddap'
odg.all_variables(server)

,count
variable,
abund_m3,2
ac_line,1
...,...
z_mean,2
zlev,6


The Axiom reader variables are for `axds_type='platform2'` not `axds_type='layer_group` since the latter are more unique grid products that don't conform well.

In [22]:
# axds 'platform2'
odg.all_variables('axds')

,count
variable,
Ammonium,23
Atmospheric Pressure: Air Pressure at Sea Level,362
...,...
Winds: at 10 m,18
pH,965


#### All available variables, sorted by count

In [23]:
# IOOS ERDDAP
server = 'http://erddap.sensors.ioos.us/erddap'
odg.search_variables(server, '')

,count
variable,
time,38331
longitude,38331
...,...
sea_surface_wave_from_direction_elw11b2t01wv_qc_tests,1
sea_water_pressure_7263arc_a,1


In [24]:
# Coastwatch ERDDAP
server = 'http://coastwatch.pfeg.noaa.gov/erddap'
odg.search_variables(server, '')

,count
variable,
time,1637
longitude,1352
...,...
haliotis_fulgens_stderr,1
observationuri,1


In [25]:
# axds 'platform2'
odg.search_variables('axds', '')

,count
variable,
Stream Height,19758
Water Surface above Datum,19489
...,...
CO2: PPM of Carbon Dioxide in Air in Dry Gas,1
Evaporation Rate,1


#### Variables search, sorted by count

In [26]:
# IOOS ERDDAP
server = 'http://erddap.sensors.ioos.us/erddap'
odg.search_variables(server, 'sal')

,count
variable,
salinity,954
salinity_qc,954
...,...
sea_water_practical_salinity_4161sc_a_qc_agg,1
sea_water_practical_salinity_10091sc_a,1


In [27]:
# Coastwatch ERDDAP
server = 'http://coastwatch.pfeg.noaa.gov/erddap'
odg.search_variables(server, 'sal')

,count
variable,
salinity,73
salt,4
...,...
stop_salinity,1
ts_salinity,1


In [28]:
# axds 'platform2'
odg.search_variables('axds', 'sal')

,count
variable,
Salinity,3204
Soil Salinity,622


#### Check variables

And finally you can check to make sure you have good variables. No news is good news in this. Reminder that you don't check for axds reader for axds_type='layer_group' because that is searched for in the database just by name as a query.

In [30]:
# IOOS ERDDAP
server = 'http://erddap.sensors.ioos.us/erddap'
odg.check_variables(server, ['salinity', 'sea_water_practical_salinity'])

In [32]:
# Coastwatch ERDDAP
server = 'http://coastwatch.pfeg.noaa.gov/erddap'
odg.check_variables(server, ['salinity', 'sea_water_salinity'])

In [33]:
# axds 'platform2'
odg.check_variables('axds', 'Salinity')

## Stations

You can search by a general station name to be searched for or by the specific database dataset_id if you know it (from performing a search previously, for example). The syntax is the same either way.

### By station name

This demonstrates the case that you know names of stations, but they might not be the names in the particular databases.

In the follow example, I use some station id's I know off the top of my head. The module will check all of the readers for the station names.

In [35]:
kwargs = {
          'approach': 'stations',
          'stations': ['8771972','SFBOFS','42020','TABS_B']
}
data = odg.Gateway(**kwargs)

In [36]:
data.dataset_ids

['noaa_nos_co_ops_8771972',
 'wmo_42020',
 'tabs_b',
 '03158b5d-f712-45f2-b05d-e4954372c1ce']

### By Dataset ID

Once we know the database dataset_ids, we can use them directly for future searches. Note that they can generally be input to the search instead of associated individually with each reader, and are input as "stations" like in the previous example.

In [37]:
kwargs = {
          'approach': 'stations',
          'stations': ['tabs_b', 'wmo_42020', 'noaa_nos_co_ops_8771972','03158b5d-f712-45f2-b05d-e4954372c1ce'],
          'erddap': {
                          'known_server': 'ioos',
          },
          'axds': {
                          'axds_type': 'layer_group',}
}
data = odg.Gateway(**kwargs)
data.dataset_ids

['tabs_b',
 'wmo_42020',
 'noaa_nos_co_ops_8771972',
 '03158b5d-f712-45f2-b05d-e4954372c1ce']

For `axds_type=='layer_group'`, you can input either the module UUID or the layer_group UUID — it generally doesn't matter unless the layer_group (a subsidiary of the module) has a different opendap url than the module or other layer_groups associated with the module.

Here we show using either the module or layer_group UUID, and inputting it in either the base level of the `kwargs` dictionary or specified for a reader. They all have the same result.

In [38]:
# Example with module uuid input as dataset_id for 'layer_group'
kwargs = {
          'approach': 'stations',
                'stations': '03158b5d-f712-45f2-b05d-e4954372c1ce',
            'axds': {
                'axds_type': 'layer_group',}
}
data = odg.Gateway(**kwargs)
data.dataset_ids

['03158b5d-f712-45f2-b05d-e4954372c1ce']

In [39]:
# Example with module uuid input as dataset_id for 'layer_group'
kwargs = {
          'approach': 'stations',
            'axds': {
                'axds_type': 'layer_group',
                'stations': '03158b5d-f712-45f2-b05d-e4954372c1ce'}
}
data = odg.Gateway(**kwargs)
data.dataset_ids

['03158b5d-f712-45f2-b05d-e4954372c1ce']

In [40]:
# Example with layer_group uuid input as station for 'layer_group'
kwargs = {
          'approach': 'stations',
              'stations': '04784baa-6be8-4aa7-b039-269f35e92e91',
            'axds': {
                'axds_type': 'layer_group',}
}
data = odg.Gateway(**kwargs)
data.dataset_ids

['03158b5d-f712-45f2-b05d-e4954372c1ce']

In [41]:
# Example with layer_group uuid input as station for 'layer_group'
kwargs = {
          'approach': 'stations',
            'axds': {
                'axds_type': 'layer_group',
              'stations': '04784baa-6be8-4aa7-b039-269f35e92e91'}
}
data = odg.Gateway(**kwargs)
data.dataset_ids

['03158b5d-f712-45f2-b05d-e4954372c1ce']

### Include Time Range

By default, the full available time range will be returned for each dataset unless the user specifies one to narrow the returned datasets in time.

Data defined in previous cell shows long time range for any of the sources you can tell there are 4 sources considered since the list in the previous code cell has 4 elements.


In [42]:
data.sources[0].kw

{'min_time': '1900-01-01', 'max_time': '2100-12-31'}

A shorter time range is shown in the following since it is specified.

In [43]:
kwargs = {
          'kw': {'min_time': '2017-1-1', 
                 'max_time': '2017-1-2'},
          'approach': 'stations',
          'stations': ['8771972']
}
data = odg.Gateway(**kwargs)
data.sources[0].kw

{'min_time': '2017-1-1', 'max_time': '2017-1-2'}

## Reader Options

### ERDDAP Reader

By default, the Data module will use `erddap` with two known servers: IOOS and Coastwatch. 

In [44]:
kwargs = {
          'kw': kw,
          'approach': 'region',
          'readers': [odg.erddap]
}
data = odg.Gateway(**kwargs)
data.sources[0].name, data.sources[1].name

('erddap_ioos', 'erddap_coastwatch')

#### Choose one known server

The user can specify to use just one of these:

In [45]:
kwargs = {
          'kw': kw,
          'approach': 'region',
          'readers': [odg.erddap],
          'erddap': {
                      'known_server': ['ioos'],  # or 'coastwatch'
          }
}
data = odg.Gateway(**kwargs)
data.sources[0].name

'erddap_ioos'

#### New ERDDAP Server

You can give the necessary information to use a different ERDDAP server.

In [46]:
kwargs = {
          'kw': kw,
          'approach': 'region',
          'readers': [odg.erddap],
            'erddap': {
                'known_server': 'ifremer',
                'protocol': 'tabledap',
                'server': 'http://www.ifremer.fr/erddap'
            }
}
data = odg.Gateway(**kwargs)

In [47]:
data.dataset_ids

['ArgoFloats-synthetic-BGC',
 'copernicus-fos',
 'ArgoFloats',
 'OceanGlidersGDACTrajectories']

### AXDS Reader

By default the Gateway class will use `axds` with two types of data: 'platform2' (like gliders) or 'layer_group' (model output, gridded products). 

In [48]:
kwargs = {
          'kw': kw,
          'approach': 'region',
          'readers': [odg.axds]
}
data = odg.Gateway(**kwargs)
data.sources[0].name, data.sources[1].name

('axds_platform2', 'axds_layer_group')

#### Specify AXDS Type

The user can specify to use just one of these:

In [49]:
kwargs = {
          'kw': kw,
          'approach': 'region',
          'readers': [odg.axds],
          'axds': {
                          'axds_type': 'platform2',  # or 'layer_group'
          }
}
data = odg.Gateway(**kwargs)
data.sources[0].name

'axds_platform2'

### Local Files

I can't remember the process by which I got these files from a portal now, but they are just meant to be sample files anyway. Hopefully this will work reasonably well with other files too.

The `region` and `stations` approach doesn't work as well with local files if the user would only be inputting filenames if they know they want to use them. It could be useful to use the approaches in the case that the user has a bunch of files somewhere or a catalog that already exists and they just want to point to that and have the code filter down. That code is not in place but could be if that is a good use case.

So it currently doesn't matter which approach is used for local files. There is a default `kw` and `region` if nothing is input and in this case that is fine since neither are used.

In [50]:
filenames = ['/Users/kthyng/Downloads/ANIMIDA_III_BeaufortSea_2014-2015/kasper-netcdf/ANIMctd14.nc',
             '/Users/kthyng/Downloads/Harrison_Bay_CTD_MooringData_2014-2015/Harrison_Bay_data/SBE16plus_01604787_2015_08_09_final.csv']

data = odg.Gateway(readers=odg.local, local={'filenames': filenames})

Can look at metadata or data

In [51]:
data.meta

,coords,download_url,time_coverage_start,lat_variable,geospatial_lon_max,lon_variable,geospatial_lon_min,time_coverage_end,variables,geospatial_lat_min,catalog_dir,time_variable,geospatial_lat_max
ANIMctd14.nc,"[time, lat, lon, pressure]",/Users/kthyng/Downloads/ANIMIDA_III_BeaufortSe...,2014-07-31T15:33:33.999999314,lat,-141.717438,lon,-152.581114,2014-08-07T21:35:54.000004381,"[station_name, sal, tem, fluoro, turbidity, PA...",69.850874,/Users/kthyng/.ocean_data_gateway/catalogs/,time,71.488255
SBE16plus_01604787_2015_08_09_final.csv,NaN,/Users/kthyng/Downloads/Harrison_Bay_CTD_Moori...,2014-08-01T12:00:05Z,NaN,-150.237,NaN,-150.237,2015-08-09T06:00:05Z,"[time, latitude, longitude, water_depth, Condu...",70.6349,/Users/kthyng/.ocean_data_gateway/catalogs/,NaN,70.6349


In [52]:
data['ANIMctd14.nc']

<xarray.Dataset>
Dimensions:              (nzmax: 1587, profile: 57)
Coordinates:
    time                 (profile) datetime64[ns] 2014-08-07T02:02:34.0000028...
    lat                  (profile) float64 71.27 71.23 71.18 ... 70.45 70.46
    lon                  (profile) float64 -152.2 -152.3 ... -145.8 -145.8
    pressure             (profile, nzmax) float64 2.187 2.399 ... -9.999e+03
Dimensions without coordinates: nzmax, profile
Data variables:
    station_name         (profile) |S12 b'1.01        ' ... b'T-XA        '
    sal                  (profile, nzmax) float64 24.85 24.85 ... -9.999e+03
    tem                  (profile, nzmax) float64 1.625 1.589 ... -9.999e+03
    fluoro               (profile, nzmax) float64 0.6842 0.7452 ... -9.999e+03
    turbidity            (profile, nzmax) float64 0.604 0.6895 ... -9.999e+03
    PAR                  (profile, nzmax) float64 9.596 9.097 ... -9.999e+03
    platform_variable    float64 9.969e+36
    instrument_variable  float64 9.969e+36
    crs                  float64 9.969e+36
Attributes: (12/35)
    Conventions:                CF-1.6
    Metadata_Conventions:       Unidata Dataset Discovery v1.0
    featureType:                profile
    cdm_data_type:              Station
    nodc_template_version:      NODC_NetCDF_Profile_Incomplete_Templete_v1.1
    standard_name_vocabulary:   NetCDF Climate and Forecast(CF) Metadata Conv...
    ...                         ...
    keywords:                   OCEAN TEMPERATURE,SALINITY,TURBIDITY,WATER PR...
    acknowledgement:            Kasper, J., CTD measurements collected from s...
    publisher_name:             Tim Whiteaker
    publisher_email:            whiteaker@utexas.edu
    publisher_url:              http://arcticstudies.org/animida_iii
    license:                    Creative Commons Attribution 3.0 United State...

In [53]:
data['SBE16plus_01604787_2015_08_09_final.csv']

,time,latitude,longitude,water_depth,Conductivity_[S/m],Pressure_[db],Temperature_ITS90_[deg C],Salinity_Practical_[PSU],Voltage0_[volts],Instrument_Time_[juliandays],flag
0,2014-08-01T12:00:05Z,70.6349,-150.237,13.0,2.495646,12.687,-1.4619,31.0905,0.3091,213.500058,0.0
1,2014-08-01T13:00:05Z,70.6349,-150.237,13.0,2.495454,12.699,-1.4595,31.0854,0.3265,213.541725,0.0
...,...,...,...,...,...,...,...,...,...,...,...
8945,2015-08-09T05:00:05Z,70.6349,-150.237,13.0,2.591448,12.777,0.3619,30.5086,0.3873,586.208391,0.0
8946,2015-08-09T06:00:05Z,70.6349,-150.237,13.0,2.585462,12.754,0.2862,30.5062,0.2441,586.250058,0.0


## Other Functionality

### Data subselection

You can pull out the data for one, several, or all of the dataset_ids found in your search, as demonstrated here.

In [54]:
kw = {'min_lon': -94,
 'max_lon': -92,
 'min_lat': 28,
 'max_lat': 30,
 'min_time': pd.Timestamp('2021-05-27'),
 'max_time': pd.Timestamp('2021-06-02')}

kwargs = {
          'kw': kw,
          'approach': 'region',
          'parallel': False,
          'readers': [odg.erddap],
          'erddap': {
                          'known_server': ['ioos'],
                           'variables': [
                                       ['sea_surface_height_above_sea_level_geoid_mllw']
                           ]
          },
}

data = odg.Gateway(**kwargs)

In [55]:
# all dataset_ids found for this search
data.dataset_ids

['noaa_nos_co_ops_8770520',
 'noaa_nos_co_ops_8770570',
 'noaa_nos_co_ops_8770475',
 'noaa_nos_co_ops_8768094',
 'noaa_nos_co_ops_8766072',
 'noaa_nos_co_ops_8770822']

#### Read in data for 1 dataset_id

Need to index with 0 to pull out the initial reader from the list (in this case there is only one).

In [56]:
data['noaa_nos_co_ops_8770822']

<xarray.Dataset>
Dimensions:                                        (time: 1458, timeseries: 1)
Coordinates:
    latitude                                       (timeseries) float64 ...
    longitude                                      (timeseries) float64 ...
  * time                                           (time) datetime64[ns] 2021...
Dimensions without coordinates: timeseries
Data variables:
    sea_surface_height_above_sea_level_geoid_mllw  (time, timeseries) float64 ...
Attributes: (12/53)
    cdm_data_type:                 TimeSeries
    cdm_timeseries_variables:      station,longitude,latitude
    contributor_email:             None,feedback@axiomdatascience.com
    contributor_name:              Gulf of Mexico Coastal Ocean Observing Sys...
    contributor_role:              funder,processor
    contributor_role_vocabulary:   NERC
    ...                            ...
    standard_name_vocabulary:      CF Standard Name Table v72
    summary:                       Timeseries data from 'Texas Point, Sabine ...
    time_coverage_end:             2021-08-19T20:00:00Z
    time_coverage_start:           2015-09-03T15:42:00Z
    title:                         Texas Point, Sabine Pass
    Westernmost_Easting:           -93.8369

In [57]:
# See what dataset_ids have been read in
data.keys()

dict_keys(['noaa_nos_co_ops_8770822'])

#### Read in data for 2 dataset_ids

In [58]:
data['noaa_nos_co_ops_8770822']
data['noaa_nos_co_ops_8770475']

<xarray.Dataset>
Dimensions:                                        (time: 1453, timeseries: 1)
Coordinates:
    latitude                                       (timeseries) float64 ...
    longitude                                      (timeseries) float64 ...
  * time                                           (time) datetime64[ns] 2021...
Dimensions without coordinates: timeseries
Data variables:
    sea_surface_height_above_sea_level_geoid_mllw  (time, timeseries) float64 ...
Attributes: (12/53)
    cdm_data_type:                 TimeSeries
    cdm_timeseries_variables:      station,longitude,latitude
    contributor_email:             None,feedback@axiomdatascience.com
    contributor_name:              Gulf of Mexico Coastal Ocean Observing Sys...
    contributor_role:              funder,processor
    contributor_role_vocabulary:   NERC
    ...                            ...
    standard_name_vocabulary:      CF Standard Name Table v72
    summary:                       Timeseries data from 'Port Arthur, TX' (ur...
    time_coverage_end:             2021-08-19T19:48:00Z
    time_coverage_start:           2015-05-05T13:00:00Z
    title:                         Port Arthur, TX
    Westernmost_Easting:           -93.93

In [59]:
# See what dataset_ids have been read in
data.keys()

dict_keys(['noaa_nos_co_ops_8770822', 'noaa_nos_co_ops_8770475'])

#### Read in data for all dataset_ids

In [60]:
for dataset_id in data.dataset_ids:
    data[dataset_id]

In [61]:
# See what dataset_ids have been read in
data.keys()

dict_keys(['noaa_nos_co_ops_8770822', 'noaa_nos_co_ops_8770475', 'noaa_nos_co_ops_8770520', 'noaa_nos_co_ops_8770570', 'noaa_nos_co_ops_8768094', 'noaa_nos_co_ops_8766072'])

### Data QC

Some quality checking of the data is possible now, along with several improvements in Dataset functionality.

#### Variable definitions

Variables to be used in the model-data comparison need to be chosen and have some basic information attached: units, and reasonable ranges for the variable in the units. These will be used to align the data and models to be sure we are making appropriate comparisons. The ranges are used for basic QC. Currently we are using the following variable information:

In [62]:
odg.var_def

{'temp': {'units': 'degree_Celsius',
  'fail_span': [-100, 100],
  'suspect_span': [-10, 40]},
 'salt': {'units': 'psu', 'fail_span': [-10, 60], 'suspect_span': [-1, 45]},
 'u': {'units': 'm/s', 'fail_span': [-10, 10], 'suspect_span': [-5, 5]},
 'v': {'units': 'm/s', 'fail_span': [-10, 10], 'suspect_span': [-5, 5]},
 'ssh': {'units': 'm', 'fail_span': [-10, 10], 'suspect_span': [-3, 3]}}

Capability in a supported package `cf-xarray` now allows for calling variables in `xarray` Datasets by user-defined names by using regular expressions to match variables definitions with these names. Here is the current version of how we want to identify variables:

In [63]:
odg.my_custom_criteria

{'ssh': {'standard_name': 'sea_surface_height$|sea_surface_elevation|sea_surface_height_above_sea_level$',
  'name': 'sea_surface_elevation|sea_surface_height_above_sea_level_geoid_mllw|zeta$'},
 'temp': {'name': 'temp$|temperature$|tem$|s.sea_water_temperature$'},
 'salt': {'standard_name': 'sea_water_salinity$|sea_water_practical_salinity$',
  'name': 'sea_water_salinity$|sea_water_practical_salinity$|salinity$|salt$|sal$|s.sea_water_practical_salinity$'},
 'u': {'standard_name': 'eastward_sea_water_velocity$|sea_water_x_velocity|surface_eastward_sea_water_velocity',
  'name': 'eastward_sea_water_velocity|sea_water_x_velocity|uo'},
 'v': {'standard_name': 'northward_sea_water_velocity$|sea_water_y_velocity|surface_northward_sea_water_velocity',
  'name': 'northward_sea_water_velocity|sea_water_y_velocity|vo'},
 'wind_speed': {'standard_name': 'wind_speed$'}}

In the following simple Dataset, a variable called "temperature" is recognized and callable by the custom name defined previously ("temp") because it compares the name and other metadata to previously-defined criteria for matching.

In [64]:
ds = xr.Dataset()
ds["temperature"] = ("dim", np.arange(10), {"units": "degree_Fahrenheit"})
ds.cf['temp']

<xarray.DataArray 'temperature' (dim: 10)>
array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])
Dimensions without coordinates: dim
Attributes:
    units:    degree_Fahrenheit

#### Units

The Datasets are ensured to have known units by converting, so comparisons with the models are appropriate.

In [65]:
ds = ds.pint.quantify()
ds.cf['temp'].pint.to('degree_Celsius')

Magnitude,[-17.777777777777743 -17.2222222222222 -16.66666666666663 -16.111111111111086 -15.555555555555543 -14.999999999999943 -14.4444444444444 -13.888888888888857 -13.333333333333314 -12.777777777777715]
Units,degree_Celsius


#### QC

Basic quality control is done for range testing of data. Currently, the output is available as datasets of flags and as a summary report (`verbose=True`).

In [66]:
filenames = ['/Users/kthyng/Downloads/ANIMIDA_III_BeaufortSea_2014-2015/kasper-netcdf/ANIMctd14.nc']
data = odg.Gateway(readers=odg.local, local={'filenames': filenames})

QC can be run on a specific dataset_id or ids too, where the dataset_ids have to be input as nested lists that match the heirarchy of the sources:

In [67]:
data.qc(dataset_ids='ANIMctd14.nc')

{'ANIMctd14.nc': <xarray.Dataset>
 Dimensions:   (nzmax: 1587, profile: 57)
 Coordinates:
     time      (profile) datetime64[ns] 2014-08-07T02:02:34.000002890 ... 2014...
     lat       (profile) float64 71.27 71.23 71.18 71.12 ... 70.38 70.45 70.46
     lon       (profile) float64 -152.2 -152.3 -152.4 ... -146.0 -145.8 -145.8
     pressure  (profile, nzmax) float64 2.187 2.399 ... -9.999e+03 -9.999e+03
 Dimensions without coordinates: nzmax, profile
 Data variables:
     tem       (profile, nzmax) float64 1.625 1.589 ... -9.999e+03 -9.999e+03
     sal       (profile, nzmax) float64 24.85 24.85 ... -9.999e+03 -9.999e+03
     tem_qc    (profile, nzmax) uint8 1 1 1 1 1 1 1 1 1 1 ... 4 4 4 4 4 4 4 4 4 4
     sal_qc    (profile, nzmax) uint8 1 1 1 1 1 1 1 1 4 1 ... 4 4 4 4 4 4 4 4 4 4
 Attributes: (12/35)
     Conventions:                CF-1.6
     Metadata_Conventions:       Unidata Dataset Discovery v1.0
     featureType:                profile
     cdm_data_type:              Station


Closer examination of the data, above, indicates that the missing values in the data are presented in the QC check as Failing, which is why there are so many values coming through as FAIL.